In [1]:
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import numpy as np
import PIL
from PIL import Image
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

PIL.Image.MAX_IMAGE_PIXELS = 933120000

In [2]:
# Define the Faster R-CNN model
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# Replace the classifier with a new one, which has a different number of classes
num_classes = 2  # background + image tiles
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

C:\Users\fhacesga\Anaconda3\envs\deeplearning\lib\site-packages\torchvision\models\_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
C:\Users\fhacesga\Anaconda3\envs\deeplearning\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [34]:
class ImageTilesDataset(torch.utils.data.Dataset):
    def __init__(self, img_path, tile_size):
        self.img = Image.open(img_path).convert('RGB')
        self.tile_size = tile_size
        self.tiles = []
        self.boxes = []
        for i in tqdm(range(0, self.img.width, self.tile_size[0])):
            for j in range(0, self.img.height, self.tile_size[1]):
                box = (i, j, i+self.tile_size[0], j+self.tile_size[1])
                tile = self.img.crop(box)
                self.tiles.append(tile)
                self.boxes.append(torch.Tensor(box))
                
        # Create a list of targets for the model
        targets = []
        for box in self.boxes:
            target = {'boxes': box.unsqueeze(0), 'labels': torch.tensor([1])}
            targets.append(target)
            
        self.targets = targets
    
    def __getitem__(self, idx):
        tile = self.tiles[idx]
        box = self.boxes[idx]
        target = self.targets[idx]
        image_tensor = torchvision.transforms.ToTensor()(tile)
        print(target)
        return image_tensor, target
    
    def __len__(self):
        return len(self.tiles)


In [37]:
# Load the image and tile size
img_path = 'D:\FloodChange\BaseRaster\BaseTest.tif'
tile_size = (128, 128)  # the size of each image tile

# Create the dataset and data loader
dataset = ImageTilesDataset(img_path, tile_size)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=True, num_workers=0)

  0%|          | 0/184 [00:00<?, ?it/s]

In [38]:
for i, ii in data_loader:
    print(len(i))
    print(ii)
    error

{'boxes': tensor([[10240.,  4480., 10368.,  4608.]]), 'labels': tensor([1])}
1
{'boxes': tensor([[[10240.,  4480., 10368.,  4608.]]]), 'labels': tensor([[1]])}


NameError: name 'error' is not defined

In [39]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.9, weight_decay=0.0005)

# Train the model
model.train()
num_epochs = 10
for epoch in range(num_epochs):
    for images, targets in data_loader:
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

{'boxes': tensor([[20352.,  4736., 20480.,  4864.]]), 'labels': tensor([1])}


TypeError: string indices must be integers

In [ ]:
# Evaluate the model on the original image
model.eval()
with torch.no_grad():
    predictions = model([torchvision.transforms.ToTensor()(img)])
    
# Process the predictions to get the bounding boxes of the image tiles
boxes = predictions[0]['boxes'].cpu().numpy()
scores = predictions[0]['scores'].cpu().numpy()
mask = scores > 0.5  # keep only the predictions with high scores
boxes = boxes[mask]

# Display the original image with the predicted bounding boxes
plt.imshow(img)
for box in boxes:
    x1, y1, x2, y2 = box.astype(int)
    plt.plot([x1, x2], [y1, y1], color='red', linewidth=2)
    plt.plot([x1, x2], [y2, y2], color='red', linewidth=2)
    plt.plot([x1, x1], [y1, y2], color='red', linewidth=2)
    plt.plot([x2, x2], [y1, y2], color='red', linewidth=2)
plt.show()